In [7]:
# https://www.benzinga.com/stock/nnvc/
import pandas as pd
from joblib import Parallel, delayed
# print(symbols)
# print(len(symbols))
import time
from selenium import webdriver
import os

import os
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
# analyst_ratings = pd.read_csv('raw_analyst_ratings.csv', index_col = 0)
# partner_headlines = pd.read_csv('raw_partner_headlines.csv', index_col = 0)
from lxml import html
from joblib import Parallel, delayed

from joblib import Parallel, delayed
from datetime import datetime

from selenium.webdriver.firefox.options import Options



In [11]:
# nasdaq_listed = pd.read_csv('nasdaq-listed-symbols_csv.csv')['Symbol'].values
# other_listed = pd.read_csv('other-listed_csv.csv')['ACT Symbol'].values
# nyse_listed = pd.read_csv('nyse-listed_csv.csv')['ACT Symbol'].values

symbols = ['SGAPY']
# for stock in nasdaq_listed:
# 	if stock not in symbols:
# 		symbols.append(stock)
# for stock in other_listed:
# 	if stock not in symbols:
# 		symbols.append(stock)
# for stock in nyse_listed:
# 	if stock not in symbols:
# 		symbols.append(stock)

symbols = ['SGAPY', 'AAPL']
if 'benzinga_scrape' not in os.listdir():
	print('Creating folder /benzinga_scrape...')
	os.mkdir('benzinga_scrape')
if 'analyst_ratings' not in os.listdir('benzinga_scrape'):
	print('Creating folder /benzinga_scrape/analyst_ratings...')
	os.mkdir('benzinga_scrape/analyst_ratings')
if 'partner_headlines' not in os.listdir('benzinga_scrape'):
	print('Creating folder /benzinga_scrape/partner_headlines')
	os.mkdir('benzinga_scrape/partner_headlines')

def get_benzinga_data(stock):
	ffox_options = Options()
	ffox_options.headless = True
	tol_amount = 5
	tols_curr = 0
	ff = webdriver.Firefox(options = ffox_options)
	try:
		ff.get('https://benzinga.com/stock/{}'.format(stock.lower()))
		time.sleep(5)
		while True:
			try:
				elem = ff.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/a/span[1]')
				ff.execute_script("arguments[0].scrollIntoView();", elem)
				time.sleep(0.3)
				elem.click()
				time.sleep(2)

			except Exception as e:
				# ff.dismiss()
				try:
					ff.find_element_by_xpath('//*[@id="onesignal-popover-cancel-button"]').click()
				except:
					pass
					# print('FAILED')
				try:
					ff.find_element_by_xpath('/html/body/div[21]/div/div/div/div/div/div/div/div[1]/div/div/div/div/div/div[4]/div/div').click()
				except:
					pass
					# print('FAILED (2)')

				# print(e)
				time.sleep(1)
				tols_curr += 1
				
				if tol_amount < tols_curr:
					break


		analyst_ratings = []
		current_index = 1
		while True:
			try:
				header = '/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/ul/li[{}]/a'.format(current_index)
				headline = ff.find_element_by_xpath(header).text
				url = ff.find_element_by_xpath(header).get_attribute('href')
				publisher = ff.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/ul/li[{}]/span[1]'.format(current_index)).text
				date = ff.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/ul/li[{}]/span[2]'.format(current_index)).text
				analyst_ratings.append([headline, url, publisher, date])
				# print(analyst_ratings[-1])
				current_index += 1
			except Exception as e:
				# print(e)
				break
		analyst_ratings = pd.DataFrame(analyst_ratings, columns = ['headline', 'url', 'publisher', 'date'])
		analyst_ratings.to_csv('benzinga_scrape/analyst_ratings/{}.csv'.format(stock))

		while True:
			try:
				elem = ff.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/a/span[1]')
				ff.execute_script("arguments[0].scrollIntoView();", elem)
				time.sleep(0.3)
				elem.click()
				time.sleep(2)

			except Exception as e:
				# ff.dismiss()
				try:
					ff.find_element_by_xpath('//*[@id="onesignal-popover-cancel-button"]').click()
				except:
					pass
					# print('FAILED')
				try:
					ff.find_element_by_xpath('/html/body/div[21]/div/div/div/div/div/div/div/div[1]/div/div/div/div/div/div[4]/div/div').click()
				except:
					pass
					# print('FAILED (2)')
				print(e)
				time.sleep(1)
				tols_curr += 1
				
				if tol_amount < tols_curr:
					break


		partner_headlines = []
		current_index = 1
		while True:
			try:
				header = '/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/ul/li[{}]/a'.format(current_index)
				headline = ff.find_element_by_xpath(header).text
				url = ff.find_element_by_xpath(header).get_attribute('href')
				publisher = ff.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/ul/li[{}]/span[1]'.format(current_index)).text
				date = ff.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/ul/li[{}]/span[2]'.format(current_index)).text
				partner_headlines.append([headline, url, publisher, date])
				# print(partner_headlines[-1])
				current_index += 1
			except Exception as e:
				# print(e)
				break
		partner_headlines = pd.DataFrame(partner_headlines, columns = ['headline', 'url', 'publisher', 'date'])
		partner_headlines.to_csv('benzinga_scrape/partner_headlines/{}.csv'.format(stock))
		ff.close()	
	except Exception as e:
		print(e)
		ff.close()


core_count = int(input('How many cores to use?: '))

print('Starting data mine...')
Parallel(core_count, 'threading', verbose = 10)(delayed(get_benzinga_data)(stock) for stock in symbols)


def get_basic_data_for_url(url):
    try:
        page = requests.get(url)
        text = page.text
        soup = BeautifulSoup(text, 'html.parser')
        date = soup.findAll('span', {'class': 'date'})[0].text
        tree = html.fromstring(page.content)
        title = tree.xpath('//*[@id="title"]')[0].text
        return title, date
    except Exception as e:
        print(e)
        return np.nan, np.nan

def wait(timestamp):
	
	print('Wait to {}'.format(timestamp))
	while True:
		current_timestamp = datetime.now()
		current_timestamp = pd.Timestamp(current_timestamp, tz = 'America/Chicago')
		if current_timestamp > timestamp:
			break
		else:
			time.sleep(5)


			



Creating folder /benzinga_scrape...
Creating folder /benzinga_scrape/analyst_ratings...
Creating folder /benzinga_scrape/partner_headlines
Starting data mine...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


'WebDriver' object has no attribute 'find_element_by_xpath'
'WebDriver' object has no attribute 'find_element_by_xpath'


[Parallel(n_jobs=8)]: Done   2 out of   2 | elapsed:   22.7s finished


In [12]:
to_scrape = []
if 'benzinga_processed' not in os.listdir(): # os.listdir() renvoie la liste des dossiers dans le repertoire
	os.mkdir('benzinga_processed')
if 'analyst_ratings_processed' not in os.listdir('benzinga_processed'):
	os.mkdir('benzinga_processed/analyst_ratings_processed')
for rating in os.listdir('benzinga_scrape/analyst_ratings'):
	if rating + '.csv' not in os.listdir('benzinga_processed/analyst_ratings_processed'):
		to_scrape.append(rating)

for analyst_rating in tqdm(to_scrape):
	path = 'benzinga_scrape/analyst_ratings/{}'.format(analyst_rating)
	data = pd.read_csv(path, index_col = 0)
	data['stock'] = analyst_rating[:-4]
	if len(data) > 0:
		if analyst_rating not in os.listdir('benzinga_processed/analyst_ratings_processed'):
			# print(len(data))
			datas = Parallel(8, 'threading', verbose = 10)(delayed(get_basic_data_for_url)(url) for url in data['url'])
			datas = pd.DataFrame(datas, columns = ['title', 'date'])
			datas.to_csv('benzinga_processed/analyst_ratings_processed/{}.csv'.format(analyst_rating))

100%|██████████| 2/2 [00:00<00:00, 164.48it/s]


In [ ]:
# get_benzinga_data('NNVC')

# print(len(to_scrape))
# wait(pd.Timestamp('2020-06-13 16:00', tz = 'America/Chicago'))

# /html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/ul/li[{}]/a - analyst ratings
# /html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/ul/li[{}]/span[1] - publisher
# /html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[7]/div/div/div[1]/ul/li[{}]/span[2] - date

# /html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/ul/li[{}]/a - partner headlines
# /html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/ul/li[{}]/span[1] - publisher
# /html/body/div[6]/div/div[2]/div[2]/div[1]/div/div[9]/div/div/div/ul/li[{}]/span[2] - date

In [14]:
import os
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from joblib import Parallel, delayed
import numpy as np
from lxml import html

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from joblib import Parallel, delayed
from datetime import datetime
import numpy as np

# Liste des symboles à scraper
# Liste des symboles à scraper

annees = []

symbols = ['APPLE-INC-4849']  # utiliser les codes Zonebourse exacts

# Dates à filtrer (format yyyy-mm-dd)
date_from = f"1990-09-27"
date_to = f"1990-09-29"

# Création du dossier news si nécessaire
if 'zonebourse_scrape' not in os.listdir():
    os.mkdir('zonebourse_scrape')
if 'news' not in os.listdir('zonebourse_scrape'):
    os.mkdir('zonebourse_scrape/news')

def get_zonebourse_news(stock):
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)

    try:
        url = f'https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/'
        driver.get(url)
        time.sleep(5)

        # ----------------------------
        # Remplir le filtre de date
        # ----------------------------
        try:
            input_from = driver.find_element(By.ID, "filter-date_from")
            input_to = driver.find_element(By.ID, "filter-date_to")

            # Effacer et remplir les dates
            input_from.clear()
            input_from.send_keys(date_from)
            input_to.clear()
            input_to.send_keys(date_to)
            input_to.send_keys(Keys.RETURN)  # Soumettre le filtre
            time.sleep(3)  # attendre le rechargement des news
        except:
            print("⚠️ Pas de filtre de date trouvé, scraping toutes les news")

        # ----------------------------
        # Scroll progressif pour charger toutes les news
        # ----------------------------
        scroll_pause = 1.5
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        current_scroll = 0
        step = 300
        while current_scroll < scroll_height:
            driver.execute_script(f"window.scrollBy(0, {step});")
            time.sleep(scroll_pause)
            current_scroll += step
            scroll_height = driver.execute_script("return document.body.scrollHeight")

        # ----------------------------
        # Récupération des news
        # ----------------------------
        news_list = []
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
        for row in rows:
            try:
                title_elem = row.find_element(By.CSS_SELECTOR, "td.w-100 div.grid a")
                date_elem = row.find_element(By.CSS_SELECTOR, "td.table-child--right span.js-date-relative")
                title = title_elem.text.strip()
                date = date_elem.get_attribute("data-utc-date")
                news_list.append([title, date])
            except:
                continue

        # Export CSV
        df_news = pd.DataFrame(news_list, columns=['title', 'date'])
        df_news.to_csv(f'zonebourse_scrape/news/{stock}_{date_from}_to_{date_to}.csv', index=False)
        print(f"✅ Done scraping news for {stock} from {date_from} to {date_to}", sep = '|')

        driver.quit()
    except Exception as e:
        print(f"Error with {stock}: {e}")
        driver.quit()

# ----------------------------
# Parallelisation
# ----------------------------
core_count = int(input('How many cores to use?: '))
Parallel(core_count, 'threading', verbose=10)(delayed(get_zonebourse_news)(stock) for stock in symbols)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


Error with APPLE-INC-4849: Message: WebDriver session does not exist, or is not active; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
InvalidSessionIDError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:472:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:581:13
assert.session@chrome://remote/content/shared/webdriver/Assert.sys.mjs:38:4
dispatch@chrome://remote/content/marionette/server.sys.mjs:315:19
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20



[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:  1.4min


[None]

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
annees = [str(a) for a in range(2025, 1989, -1)]

symbols = [
    'NVIDIA-CORPORATION-57355629',
    'MICROSOFT-CORPORATION-4835',
    'APPLE-INC-4849',
    'ALPHABET-INC-24203373',
    'META-PLATFORMS-INC-10547141',
    'TESLA-INC-6344549',
    'BERKSHIRE-HATHAWAY-INC-11915',
    'BROADCOM-INC-42668543'
]

os.makedirs("zonebourse_scrape/news", exist_ok=True)

def get_zonebourse_news(stock, year):
    date_from = f"{year}-01-01"
    date_to = f"{year}-12-31"
    news_list = []

    options = Options()
    options.headless = True  # passe à False si tu veux voir les fenêtres
    driver = webdriver.Firefox(options=options)

    try:
        url = f"https://www.marketscreener.com/quote/stock/{stock}/news/"
        driver.get(url)
        time.sleep(5)

        try:
            input_from = driver.find_element(By.ID, "filter-date_from")
            input_to = driver.find_element(By.ID, "filter-date_to")
            input_from.clear()
            input_from.send_keys(date_from)
            input_to.clear()
            input_to.send_keys(date_to)
            input_to.send_keys(Keys.RETURN)
            time.sleep(3)
        except:
            print("⚠️ Pas de filtre de date trouvé, scraping toutes les news")

        scroll_pause = 1.5
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(scroll_pause)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
        for row in rows:
            try:
                title_elem = row.find_element(By.CSS_SELECTOR, "td.w-100 div.grid a")
                date_elem = row.find_element(By.CSS_SELECTOR, "td.table-child--right span.js-date-relative")
                title = title_elem.text.strip()
                date = date_elem.get_attribute("data-utc-date")
                news_list.append([title, date])
            except:
                continue

        df_news = pd.DataFrame(news_list, columns=["title", "date"])
        filepath = f"zonebourse_scrape/news/{stock}_{year}.csv"
        df_news.to_csv(filepath, index=False)
        print(f"✅ {stock} | {year} | {len(news_list)} news trouvées")
    except Exception as e:
        print(f"❌ Erreur sur {stock} {year} : {e}")
    finally:
        driver.quit()

tasks = [(stock, year) for stock in symbols for year in annees]

n_cores = multiprocessing.cpu_count()
print(f"🚀 Lancement du scraping en parallèle avec {n_cores} threads...")

# 🔑 UTILISATION DE THREADING POUR WINDOWS
Parallel(n_jobs=n_cores, backend="threading", verbose=10)(
    delayed(get_zonebourse_news)(stock, year) for stock, year in tasks
)

print("🎯 Scraping terminé ! Résultats dans zonebourse_scrape/news/")

🚀 Lancement du scraping en parallèle avec 16 threads...


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.


✅ NVIDIA-CORPORATION-57355629 | 2011 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2025 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2017 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2012 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2020 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2014 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2013 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2024 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2019 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2023 | 50 news trouvées


[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:  2.3min


✅ NVIDIA-CORPORATION-57355629 | 2016 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2021 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2018 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2009 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2006 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2005 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2008 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2007 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2010 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2001 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2004 | 50 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2022 | 50 news trouvées


...

✅ APPLE-INC-4849 | 2017 | 50 news trouvées
✅ APPLE-INC-4849 | 2004 | 50 news trouvées
✅ APPLE-INC-4849 | 2006 | 50 news trouvées
✅ APPLE-INC-4849 | 2008 | 50 news trouvées
✅ APPLE-INC-4849 | 2002 | 50 news trouvées
✅ APPLE-INC-4849 | 2007 | 50 news trouvées
✅ APPLE-INC-4849 | 1998 | 50 news trouvées
✅ APPLE-INC-4849 | 1997 | 50 news trouvées
✅ APPLE-INC-4849 | 2001 | 50 news trouvées
✅ APPLE-INC-4849 | 1995 | 50 news trouvées
✅ APPLE-INC-4849 | 2005 | 50 news trouvées
✅ APPLE-INC-4849 | 1996 | 50 news trouvées
✅ APPLE-INC-4849 | 2000 | 50 news trouvées
✅ APPLE-INC-4849 | 1994 | 50 news trouvées


[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed: 11.5min


✅ ALPHABET-INC-24203373 | 2022 | 50 news trouvées
✅ ALPHABET-INC-24203373 | 2021 | 50 news trouvées
✅ ALPHABET-INC-24203373 | 2016 | 50 news trouvées✅ ALPHABET-INC-24203373 | 2017 | 50 news trouvées

✅ APPLE-INC-4849 | 1991 | 50 news trouvées
✅ ALPHABET-INC-24203373 | 2023 | 50 news trouvées
❌ Erreur sur APPLE-INC-4849 1999 : HTTPConnectionPool(host='localhost', port=50085): Read timed out. (read timeout=120)
✅ ALPHABET-INC-24203373 | 2025 | 50 news trouvées
✅ APPLE-INC-4849 | 1992 | 50 news trouvées


KeyboardInterrupt: 

✅ ALPHABET-INC-24203373 | 2020 | 50 news trouvées
✅ APPLE-INC-4849 | 1993 | 50 news trouvées
✅ ALPHABET-INC-24203373 | 2024 | 50 news trouvées✅ APPLE-INC-4849 | 1990 | 50 news trouvées

✅ ALPHABET-INC-24203373 | 2019 | 50 news trouvées✅ APPLE-INC-4849 | 2003 | 50 news trouvées

✅ ALPHABET-INC-24203373 | 2018 | 50 news trouvées
❌ Erreur sur ALPHABET-INC-24203373 2014 : Message: Tried to run command without establishing a connection; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception

❌ Erreur sur ALPHABET-INC-24203373 2013 : Message: Tried to run command without establishing a connection; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception

❌ Erreur sur ALPHABET-INC-24203373 2012 : Message: Tried to run command without establishing a connection; For documentation on this error, please visit: https://www

In [6]:
import os
import time
import pandas as pd
import multiprocessing
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from joblib import Parallel, delayed

# Années à scraper
annees = [str(a) for a in range(2025, 1989, -1)]

# Liste des symboles à scraper
symbols = [
    'NVIDIA-CORPORATION-57355629',
    'MICROSOFT-CORPORATION-4835',
    'APPLE-INC-4849',
    'ALPHABET-INC-24203373',
    'META-PLATFORMS-INC-10547141',
    'TESLA-INC-6344549',
    'BERKSHIRE-HATHAWAY-INC-11915',
    'BROADCOM-INC-42668543'
]

os.makedirs("zonebourse_scrape/news", exist_ok=True)

def get_zonebourse_news(stock, year, max_retries=4):
    date_from = f"{year}-01-01"
    date_to = f"{year}-12-31"
    news_list = []

    for attempt in range(max_retries):
        options = Options()
        options.headless = True  # Passe à False pour voir les fenêtres
        driver = webdriver.Firefox(options=options)

        try:
            url = f"https://www.marketscreener.com/quote/stock/{stock}/news/"
            driver.get(url)
            time.sleep(3)

            # Appliquer le filtre de dates
            try:
                input_from = driver.find_element(By.ID, "filter-date_from")
                input_to = driver.find_element(By.ID, "filter-date_to")
                input_from.clear()
                input_from.send_keys(date_from)
                input_to.clear()
                input_to.send_keys(date_to)
                input_to.send_keys(Keys.RETURN)

                # Attendre que les news soient mises à jour
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "tbody tr"))
                )
                time.sleep(2)
            except:
                print(f"⚠️ {stock} {year} : Pas de filtre de date trouvé")

            # Scroll progressif
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(3)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Récupération des news
            rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
            for row in rows:
                try:
                    title_elem = row.find_element(By.CSS_SELECTOR, "td.w-100 div.grid a")
                    date_elem = row.find_element(By.CSS_SELECTOR, "td.table-child--right span.js-date-relative")
                    title = title_elem.text.strip()
                    date = date_elem.get_attribute("data-utc-date")
                    news_list.append([title, date])
                except:
                    continue

            if len(news_list) == 0 and attempt < max_retries - 1:
                print(f"🔄 {stock} {year} : 0 news, on réessaie ({attempt+1}/{max_retries})...")
                driver.quit()
                time.sleep(3)
                continue  # retenter
            break  # sortie si on a récupéré des news

        except Exception as e:
            print(f"❌ Erreur sur {stock} {year} : {e}")
        finally:
            driver.quit()

    # Sauvegarde CSV même si vide (utile pour savoir que l'année a été traitée)
    df_news = pd.DataFrame(news_list, columns=["title", "date"])
    filepath = f"zonebourse_scrape/news/{stock}_{year}.csv"
    df_news.to_csv(filepath, index=False)
    print(f"✅ {stock} | {year} | {len(news_list)} news trouvées")

tasks = [(stock, year) for stock in symbols for year in annees]

# ✅ limiter le nombre de cœurs utilisés (2 ou 4 est souvent suffisant)
n_cores = min(4, multiprocessing.cpu_count())
print(f"🚀 Lancement du scraping en parallèle avec {n_cores} threads...")

Parallel(n_jobs=n_cores, backend="threading", verbose=10)(
    delayed(get_zonebourse_news)(stock, year) for stock, year in tasks
)

print("🎯 Scraping terminé ! Résultats dans zonebourse_scrape/news/")


🚀 Lancement du scraping en parallèle avec 4 threads...


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


✅ NVIDIA-CORPORATION-57355629 | 2023 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2025 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2024 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2022 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2021 | 25 news trouvées


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   37.2s


✅ NVIDIA-CORPORATION-57355629 | 2020 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2019 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2018 | 25 news trouvées
⚠️ NVIDIA-CORPORATION-57355629 2014 : Pas de filtre de date trouvé
❌ Erreur sur NVIDIA-CORPORATION-57355629 2014 : Message: Tried to run command without establishing a connection; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception

✅ NVIDIA-CORPORATION-57355629 | 2017 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2016 | 25 news trouvées


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   58.5s


✅ NVIDIA-CORPORATION-57355629 | 2015 | 25 news trouvées


KeyboardInterrupt: 

✅ NVIDIA-CORPORATION-57355629 | 2013 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2014 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2011 | 25 news trouvées
✅ NVIDIA-CORPORATION-57355629 | 2012 | 25 news trouvées


In [ ]:
# ==== GDELT -> DF avec Multiple Batches ====================================
from gdeltdoc import GdeltDoc, Filters, near, repeat
import pandas as pd

def get_multiple_batches(num_batches=150):
    gd = GdeltDoc()
    all_articles = []
    from datetime import datetime, timedelta
    import time
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2025, 9, 20)
    total_days = (end_date - start_date).days
    days_per_batch = total_days // num_batches
    current_date = start_date
    for i in range(num_batches):
        if i == num_batches - 1:
            period_end = end_date
        else:
            period_end = current_date + timedelta(days=days_per_batch)
        f = Filters(
            start_date=current_date.strftime("%Y-%m-%d"),
            end_date=period_end.strftime("%Y-%m-%d"),
            num_records=250,
            country = "US",
            repeat = repeat(2, "oil"),
            language="ENGLISH",
            keyword = "Brent"
            # domain=["bbc.co.uk", "bloomberg.com", "theguardian.com", "ft.com","economist.com"]
        )
        try:
            df_batch = gd.article_search(f)
            if not df_batch.empty:
                all_articles.append(df_batch)
                print(f"Batch {i+1} ({current_date.strftime('%Y-%m-%d')} à {period_end.strftime('%Y-%m-%d')}): {len(df_batch)} articles")
        except Exception as e:
            print(f"Erreur batch {i+1}: {e}")
        current_date = period_end
        time.sleep(1)
    if all_articles:
        final_df = pd.concat(all_articles, ignore_index=True)
        final_df = final_df.drop_duplicates(subset=['url'], keep='first')
        print(f"Total final après suppression doublons: {len(final_df)} articles")
        return final_df
    return pd.DataFrame()

news = get_multiple_batches(num_batches=200)

# news = news[['title', 'seendate']]
# news.columns = ['title', 'date']
# # Changer de format de 20170117T183000Z = 
# news['date'] = pd.to_datetime(news['date'], format = '%Y%m%dT%H%M%SZ')
# news['hour'] = news['date'].dt.strftime('%H:%M:%S')
# news['date'] = news['date'].dt.date
# news['score'] = news['title'].apply(finbert_sentiment)

news.to_csv("news.csv", sep = '|')

In [ ]:
import requests
import pandas as pd
import time

from gdeltdoc import GdeltDoc, Filters, repeat
import pandas as pd
import time
from datetime import datetime, timedelta



In [21]:


# Exemple : dictionnaire avec mots-clés et période d'appartenance au S&P 500
companies = {
    
    "SP500": {
        "terms": [
        # Indice S&P 500
        "SP500", "S&P 500", "S and P 500", "Standard & Poor's 500", "SP-500", "SPX Index", "S&P500", "sp 500", "SPX",
        
        # Marchés financiers US en général
        "stock market", "financial market", "equity market", "Wall Street", "Dow Jones", "NASDAQ", "NYSE",
        
        # Entreprises / secteurs
        "tech sector", "financial sector", "energy sector", "consumer sector", "industrials sector",
        
        # Économie US
        "US economy", "United States economy", "GDP", "economic growth", "recession", "unemployment", "Treasury yields",
        "inflation", "CPI", "PCE", "interest rates", "Fed", "Federal Reserve", "monetary policy", "US economy",
        "fiscal policy", "budget deficit", "trade deficit", "US Treasury", "government bonds", "10-year yield",
            # "jobless claims",
        
        # Événements macro / géopolitiques
        "recession fears", "economic crisis", "market volatility", "stock crash", "financial crisis", "quantitative easing"
            # ,"QE", "tightening", "stimulus package", "tax reform", "regulatory changes"
        ],
        "start_date": "1990-01-01",   # date d'entrée approximative
        "end_date": "2025-12-31"
    }
    # "AAPL": {
    #     "terms": ["Apple Inc", "AAPL"],
    #     "start_date": "2025-01-01",   # date d'entrée approximative
    #     "end_date": "2025-12-31"
    # }
    # "MSFT": {
    #     "terms": ["Microsoft Corp", "MSFT"],
    #     "start_date": "2025-06-01",   # date d'entrée approximative
    #     "end_date": "2025-12-31"
    # }
    # "XOM": {
    #     "terms": ["Exxon Mobil", "Exxon", "XOM"],
    #     "start_date": "1957-03-04",   # Exxon est là depuis la création du S&P 500
    #     "end_date": "2025-12-31"
    # }
}

# Paramètres de recherche
BASE_URL = "https://archive.org/advancedsearch.php"
LANG_FILTER = "language:(eng)"

results = []

for ticker, info in companies.items():
    terms = info["terms"]
    start_date = info["start_date"]
    end_date = info["end_date"]

    # Construire la requête
    query_terms = " OR ".join([f'"{t}"' for t in terms])
    date_filter = f"date:[{start_date} TO {end_date}]"
    query = f"({query_terms}) AND {date_filter} AND {LANG_FILTER}"

    print(f"🔎 Recherche pour {ticker}: {query}")

    page = 1
    while True:
        params = {
            "q": query,
            "fl[]": ["identifier", "title", "creator", "date", "language", "description"],
            "sort[]": "date desc",
            "rows": 100000000,
            "page": page,
            "output": "json"
        }

        try:
            r = requests.get(BASE_URL, params=params)
            r.raise_for_status()
            data = r.json()
        except Exception as e:
            print(f"⚠️ Erreur pour {ticker} page {page}: {e}")
            break

        docs = data.get("response", {}).get("docs", [])
        if not docs:
            break

        for doc in docs:
            results.append({
                "ticker": ticker,
                "title": doc.get("title", ""),
                "date": doc.get("date", ""),
                "creator": doc.get("creator", ""),
                "language": doc.get("language", ""),
                "collection": doc.get("collection", ""),
                "subject": doc.get("subject", ""),
                "rights": doc.get("rights", ""),
                "description": doc.get("description", ""),
                "format": doc.get("format", ""),
                "publicdate": doc.get("publicdate", ""),
                "link": f"https://archive.org/details/{doc.get('identifier', '')}"
            })

        page += 1
        time.sleep(1)  # éviter de surcharger l'API

# Sauvegarde en CSV
df = pd.DataFrame(results)
df.to_csv("sp500_archive_news.csv", index=False, encoding="utf-8-sig", sep = '|')

print(f"✅ Export terminé : {len(df)} résultats enregistrés dans sp500_archive_news.csv")


🔎 Recherche pour SP500: ("SP500" OR "S&P 500" OR "S and P 500" OR "Standard & Poor's 500" OR "SP-500" OR "SPX Index" OR "S&P500" OR "sp 500" OR "SPX" OR "stock market" OR "financial market" OR "equity market" OR "Wall Street" OR "Dow Jones" OR "NASDAQ" OR "NYSE" OR "tech sector" OR "financial sector" OR "energy sector" OR "consumer sector" OR "industrials sector" OR "US economy" OR "United States economy" OR "GDP" OR "economic growth" OR "recession" OR "unemployment" OR "Treasury yields" OR "inflation" OR "CPI" OR "PCE" OR "interest rates" OR "Fed" OR "Federal Reserve" OR "monetary policy" OR "US economy" OR "fiscal policy" OR "budget deficit" OR "trade deficit" OR "US Treasury" OR "government bonds" OR "10-year yield" OR "recession fears" OR "economic crisis" OR "market volatility" OR "stock crash" OR "financial crisis" OR "quantitative easing") AND date:[1990-01-01 TO 2025-12-31] AND language:(eng)
⚠️ Erreur pour SP500 page 2: Expecting value: line 1 column 2 (char 1)
✅ Export termin

🔎 Recherche pour SP500: ("SP500" OR "S&P 500" OR "S and P 500" OR "Standard & Poor's 500" OR "SP-500" OR "SPX Index" OR "S&P500" OR "sp 500" OR "SPX" OR "stock market" OR "financial market" OR "equity market" OR "Wall Street" OR "Dow Jones" OR "NASDAQ" OR "NYSE" OR "tech sector" OR "financial sector" OR "energy sector" OR "consumer sector" OR "industrials sector" OR "US economy" OR "United States economy" OR "GDP" OR "economic growth" OR "recession" OR "unemployment" OR "Treasury yields" OR "inflation" OR "CPI" OR "PCE" OR "interest rates" OR "Fed" OR "Federal Reserve" OR "monetary policy" OR "US economy" OR "fiscal policy" OR "budget deficit" OR "trade deficit" OR "US Treasury" OR "government bonds" OR "10-year yield" OR "recession fears" OR "economic crisis" OR "market volatility" OR "stock crash" OR "financial crisis" OR "quantitative easing") AND date:[1990-01-01 TO 2025-12-31] AND language:(eng)

⚠️ Erreur pour SP500 page 2: Expecting value: line 1 column 2 (char 1)

✅ Export terminé : 233388 résultats enregistrés dans sp500_archive_news.csv

In [ ]:
# ==== GDELT -> DF avec Multiple Batches ====================================
# Liste de mots-clés importants liés au S&P500 et macro US
KEYWORDS = [
        # Indice S&P 500
        "SP500", "S&P 500", "S and P 500", "Standard & Poor's 500", "SP-500", "SPX Index", "S&P500", "sp 500", "SPX",
        
        # Marchés financiers US en général
        "stock market", "financial market", "equity market", "Wall Street", "Dow Jones", "NASDAQ", "NYSE",
        
        # Entreprises / secteurs
        "tech sector", "financial sector", "energy sector", "consumer sector", "industrials sector",
        
        # Économie US
        "US economy", "United States economy", "GDP", "economic growth", "recession", "unemployment", "Treasury yields",
        "inflation", "CPI", "PCE", "interest rates", "Fed", "Federal Reserve", "monetary policy", "US economy",
        "fiscal policy", "budget deficit", "trade deficit", "US Treasury", "government bonds", "10-year yield",
            # "jobless claims",
        
        # Événements macro / géopolitiques
        "recession fears", "economic crisis", "market volatility", "stock crash", "financial crisis", "quantitative easing"
            # ,"QE", "tightening", "stimulus package", "tax reform", "regulatory changes"
        ]

def get_multiple_batches(num_batches=150, keywords=KEYWORDS):
    gd = GdeltDoc()
    all_articles = []

    # Période
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2025, 9, 20)
    total_days = (end_date - start_date).days
    days_per_batch = total_days // num_batches
    current_date = start_date

    # Découpage par batch
    for i in range(num_batches):
        if i == num_batches - 1:
            period_end = end_date
        else:
            period_end = current_date + timedelta(days=days_per_batch)

        for kw in keywords:
            f = Filters(
                start_date=current_date.strftime("%Y-%m-%d"),
                end_date=period_end.strftime("%Y-%m-%d"),
                num_records=250,
                country="US",
                language="ENGLISH",
                keyword=kw
                # domain=["bloomberg.com", "ft.com", "reuters.com"]
            )
            try:
                df_batch = gd.article_search(f)
                if not df_batch.empty:
                    df_batch["keyword"] = kw
                    all_articles.append(df_batch)
                    print(f"Batch {i+1} ({current_date.strftime('%Y-%m-%d')} à {period_end.strftime('%Y-%m-%d')}), keyword={kw}: {len(df_batch)} articles")
            except Exception as e:
                print(f"⚠️ Erreur batch {i+1}, keyword={kw}: {e}")
            time.sleep(1)

        current_date = period_end

    # Concaténer tous les résultats
    if all_articles:
        final_df = pd.concat(all_articles, ignore_index=True)
        final_df = final_df.drop_duplicates(subset=['url'], keep='first')
        print(f"\n✅ Total final après suppression doublons: {len(final_df)} articles")
        return final_df
    return pd.DataFrame()

# Exécution
news = get_multiple_batches(num_batches=200)

# Formatage optionnel
if not news.empty:
    news = news[['title', 'seendate', 'url', 'domain', 'keyword']]
    news.columns = ['title', 'date', 'url', 'source', 'keyword']
    news['date'] = pd.to_datetime(news['date'], format='%Y%m%dT%H%M%SZ', errors='coerce')

    # Sauvegarde CSV
    news.to_csv("gdelt_news.csv", sep="|", index=False, encoding="utf-8-sig")
    print("💾 Export terminé : gdelt_news.csv")


In [23]:
creator = pd.read_csv("sp500_archive_news.csv", sep= '|')['creator'].value_counts()

In [55]:
count = pd.DataFrame(creator)
(count[count['count'] > 3].index).to_series().to_csv("result.txt", index=False, header=False, sep = '|')
count

,count
creator,
Australian Bureau of Statistics,2282
Jack Saturday,503
USAWATCHDOG.COM,471
"Missouri Department of Higher Education & Workforce Development, Missouri Economic Research and Information Center",205
The Horror of Babylon,203
...,...
National Conference on Business Ethics (8th : 1989 : Bentley College),1
"Gitlow, Howard S",1
"['Schraff, Anne E', 'Feinberg, Barbara Silberdick']",1


In [6]:
import pandas as pd

df = pd.read_csv("sp500_archive_news.csv", sep= '|')

# liste des sources pertinentes
relevant_sources = [
    # Médias économiques
    "The Wall Street Journal", "Wall Street Journal", "CNBC", "CNBC, Jim Cramer", "CNBC, James Cramer",
    "Fox Business", "BBC News", "BBC", "ABC News", "PBS", "Al Jazeera English", "Al Jazeera",
    "Federal Reserve - Federal Open Market Committee (FOMC)",
    "Federal Reserve Bank of New York",
    "Federal Reserve Board of Governors",
    "U.S. Treasury Department",
    "United States. Congress. House. Committee on the Budget",
    "United States. Congress. Senate. Committee on Banking, Housing, and Urban Affairs",
    "United States. Congress. Joint Economic Committee",
    "Congressional Research Service",
    "OECD",
    "World Bank",
    "European Central Bank",
    "Wall Street Journal",
    "The Wall Street Journal",
    "CNBC",
    "CNBC, Jim Cramer",
    "Fox Business",
    "BBC News",
    "BBC",
    "ABC News",
    "Al Jazeera English",
    "PBS",
    "Paul Krugman",
    "Stiglitz, Joseph E",
    "Shiller, Robert J",
    "Mankiw, N. Gregory",
    "Blinder, Alan S",
    "Keynes, John Maynard, 1883-1946",
    "Blanchard, Olivier",
    "Malkiel, Burton Gordon",
    "Siegel, Jeremy J",
    "Sorkin, Andrew Ross",
    "Tett, Gillian",
    "Wolf, Martin, 1946-",
    "Eichengreen, Barry J",
    "Mishkin, Frederic S",
    "Michael Lewis",
    "Berkshire Hathaway",
    "Microsoft",
    "Apple Inc.",
    "Amazon.com",
    "Coca-Cola Company",
    "McDonald's Corporation",
    "Walt Disney Company",
    "ExxonMobil",
    "Chevron Corp.",
    "FedEx Corporation",
    "Caterpillar",
    "Halliburton Company",
    "BlackRock",
    "Starbucks",
    "Beyond Meat, Inc.",
    "Las Vegas Sands Corp.",
    "Wynn Resorts Limited",
    "Brown-Forman Corporation",
    "Salesforce.com",
    "Altria Group, Inc.",
    "Anheuser-Busch InBev SA/NV",
    "Monster Beverage Corporation",
    "YUM! Brands",
    "United Rentals",
    
    # Institutions US
    "Federal Reserve Bank of New York", "Federal Reserve - Federal Open Market Committee (FOMC)",
    "Federal Reserve Board of Governors", "U.S. Treasury Department",
    "United States. Congress. Joint Economic Committee",
    "United States. Congress. Senate. Committee on Banking, Housing, and Urban Affairs",
    "United States. Congress. Senate. Committee on Finance",
    "United States. Congress. House. Committee on Banking, Finance, and Urban Affairs",
    "United States. Congress. House. Committee on Ways and Means",
    "Supreme Court of the United States",  # décisions juridiques à fort impact économique
    
    # Organisations internationales
    "Organisation for Economic Co-operation and Development", "OECD", "World Bank",
    
    # Grandes entreprises US cotées (exemples du S&P 500 présents dans ta liste)
    "Berkshire Hathaway", "Microsoft", "Apple Inc.", "Amazon.com", "Walt Disney Company",
    "The Coca-Cola Company", "McDonald's Corporation", "Chevron Corp.", "ExxonMobil",
    "Caterpillar", "FedEx Corporation", "Southwest Airlines Co.", "BlackRock",
    "Salesforce.com", "Halliburton Company", "Wynn Resorts Limited",
    
    # Auteurs / économistes influents
    "Krugman, Paul R", "Stiglitz, Joseph E", "Shiller, Robert J", "Mankiw, N. Gregory",
    "Blinder, Alan S", "Greenspan, Alan, 1926-", "Eichengreen, Barry J",
    "Siegel, Jeremy J", "Taylor, John B", "Varoufakis, Yanis, author",
]

# filtrer le DataFrame



In [7]:
filtered_df = df[df["creator"].isin(relevant_sources)]

filtered_df

,ticker,title,date,creator,language,collection,subject,rights,description,format,publicdate,link
56,SP500,The Man Leading Christian Nationalism’s Rise U...,2025-09-26T00:00:00Z,The Wall Street Journal,eng,NaN,NaN,NaN,Original Title: The Man Leading Christian Nati...,NaN,NaN,https://archive.org/details/Zlz_0kwno0c
57,SP500,The Big Problem With Mamdani’s City-Run Grocer...,2025-09-26T00:00:00Z,CNBC,eng,NaN,NaN,NaN,Original Title: The Big Problem With Mamdani’s...,NaN,NaN,https://archive.org/details/mRpS3n_Y68k
80,SP500,The Complex Tech Behind Growing Diamonds in Mi...,2025-09-25T00:00:00Z,The Wall Street Journal,eng,NaN,NaN,NaN,Original Title: The Complex Tech Behind Growin...,NaN,NaN,https://archive.org/details/SPAMTs6Ug20
90,SP500,This Poll Exposes a Gender Gap That May Shape ...,2025-09-24T00:00:00Z,The Wall Street Journal,eng,NaN,NaN,NaN,Original Title: This Poll Exposes a Gender Gap...,NaN,NaN,https://archive.org/details/aiGQja2yRYo
98,SP500,Why So Many Americans Are Clinging To Their Jo...,2025-09-24T00:00:00Z,CNBC,eng,NaN,NaN,NaN,Original Title: Why So Many Americans Are Clin...,NaN,NaN,https://archive.org/details/MG1I8fQp43Y
...,...,...,...,...,...,...,...,...,...,...,...,...
233151,SP500,Annual report - Federal Reserve Bank of New York,1990-01-01T00:00:00Z,Federal Reserve Bank of New York,eng,NaN,NaN,NaN,"['v. ; 23 cm', 'Title varies slightly', 'First...",NaN,NaN,https://archive.org/details/annualreportfede00...
233268,SP500,The Walt Disney Co. 1989 Annual Report,1990-01-01T00:00:00Z,Walt Disney Company,eng,NaN,NaN,NaN,"The 1989 annual report & Form 10-K, and Chairm...",NaN,NaN,https://archive.org/details/walt-disney-compan...
233317,SP500,City Disc LaserDisc DD86 Disc Set 1,1990-01-01T00:00:00Z,BBC,eng,NaN,NaN,NaN,This is an RF copy of the City LaserDisc creat...,NaN,NaN,https://archive.org/details/city-disc-laserdis...
233349,SP500,A random walk down Wall Street : including a l...,1990-01-01T00:00:00Z,"Malkiel, Burton Gordon",eng,NaN,NaN,NaN,"['440 p. : 24 cm', 'Includes bibliographical r...",NaN,NaN,https://archive.org/details/randomwalkdownwa00...


In [8]:
df

,ticker,title,date,creator,language,collection,subject,rights,description,format,publicdate,link
0,SP500,"Varney & Company : FBC : September 30, 2025 10...",2025-09-30T00:00:00Z,NaN,eng,NaN,NaN,NaN,Wall Street news.,NaN,NaN,https://archive.org/details/FBC_20250930_14000...
1,SP500,"Varney & Company : FBC : September 30, 2025 11...",2025-09-30T00:00:00Z,NaN,eng,NaN,NaN,NaN,Wall Street news.,NaN,NaN,https://archive.org/details/FBC_20250930_15000...
2,SP500,Business Today - NYSE Opening Bell : BBCNEWS :...,2025-09-30T00:00:00Z,NaN,eng,NaN,NaN,NaN,Bringing the latest business news and up-to-da...,NaN,NaN,https://archive.org/details/BBCNEWS_20250930_1...
3,SP500,Bloomberg Surveillance : BLOOMBERG : September...,2025-09-30T00:00:00Z,NaN,eng,NaN,NaN,NaN,"Jonathan, Lisa and Annmarie survey the economy...",NaN,NaN,https://archive.org/details/BLOOMBERG_20250930...
4,SP500,"Varney & Company : FBC : September 30, 2025 9:...",2025-09-30T00:00:00Z,NaN,eng,NaN,NaN,NaN,Wall Street news.,NaN,NaN,https://archive.org/details/FBC_20250930_13000...
...,...,...,...,...,...,...,...,...,...,...,...,...
233383,SP500,Economic growth & employment structure : a stu...,1990-01-01T00:00:00Z,"Ghose, Ajit Kumar, 1947-",eng,NaN,NaN,NaN,"['xi, 99 p. ; 24 cm', 'Includes bibliographica...",NaN,NaN,https://archive.org/details/economicgrowthem00...
233384,SP500,Colorado,1990-01-01T00:00:00Z,Silmarils,eng,NaN,NaN,NaN,Colorado (1990)(Silmarils).st,NaN,NaN,https://archive.org/details/Colorado_1990_Silm...
233385,SP500,"Hunt for Red October, The",1990-01-01T00:00:00Z,Grandslam,eng,NaN,NaN,NaN,"Hunt for Red October, The (1990)(Grandslam)[cr...",NaN,NaN,https://archive.org/details/Hunt_for_Red_Octob...
233386,SP500,"Immortal, The",1990-01-01T00:00:00Z,Electronic Arts,eng,NaN,NaN,NaN,"Immortal, The (1990)(Electronic Arts)[cr Letha...",NaN,NaN,https://archive.org/details/Immortal_The_1990_...


In [68]:
import re

exclude_keywords = [
    # Religion / spirituel
    "Church", "Unitarian", "UU", "Sermon", "Faith", "Bible", "Ministr",
    "Pastor", "Rev.", "Prophet", "JD Farag", "Younce",

    # Conspiration / extrême / désinformation
    "Infowars", "Corbett", "TruNews", "Watchdog", "lookintoit", "Unlimited Hangout",
    "Deep State", "CODOH", "Holocaust Denial", "Truth", "Conspiracy", "Awakening",
    "Alternative Media", "UK Column", "Unsafe Space", "VirusTruth", "QAnon",
    "Geo Godley", "Tucker Carlson", "Robert F Kennedy", "E Michael Jones",
    "Stew Peters", "Team Occulted", "Hitler", "Goebbels", "Wehrwolf", "MaeBrussell",
    "Bitchute", "Corona Ausschuss", "Malcolm Roberts", "Carlo Mattogno",

    # Médias / figures très polarisés
    "Jordan Peterson", "Candace Owens", "David Pakman", "Democracy Now", "OnePath Network",
    "TRT World", "RT ", "Russia Today", "Sputnik", "TASS", "Kremlin",

    # Formats amateurs / auto-promo
    "Podcast", "Radio", "Blog", "YouTube", "Soundcloud", "Substack",
    "Studio", "Productions", "Network", "Press", "Publishing", "AuthorScript",
    "puzzle in a thunderstorm", "LLC", "Inc.", "Enterprises", "Consultancy",
    "Agency", "Solutions", "Marketing",

    # Non-pertinent / bruit (artistes, pseudos, divertissement)
    "Cats", "Funny", "Comedy", "Humor", "Satire", "Cartoon", "Meme", "Gaming",
    "Wrestling", "Anime", "Portrait", "Dog", "Books", "Novels", "Art",
    "Creative", "Coffee", "Music", "Butcher", "Netflix", "Clips",

    # Finance douteuse / auto-promo
    "ProfitHills", "Trading", "Forex", "Crypto", "Course", "Mentorship", "Motivational",

    # Pseudos / noms farfelus
    "Skibidi", "Ghosty", "kittyfiya", "noon2025", "Mr ", "Mrs ", "DrWeb",
    "Mr.noon", "brïde", "MadLad", "Purplecade", "Breanna Bob", "Otto Maddox",
    "Disturbed Rhys", "Allure_Queen", "Commander Ikari", "SunlightAFA", "feverdreamerteam",


    # Religion / spirituel controversé
    "Church", "Unitarian", "UU", "Sermon", "Faith", "Bible", "Ministr",
    "Pastor", "Rev.", "Prophet", "JD Farag", "Younce", "Pastor Larry Hughes", "Archbishop Carlo Maria Viganò",

    # Conspiration / extrême / désinformation
    "Infowars", "Corbett", "TruNews", "Watchdog", "lookintoit", "Unlimited Hangout",
    "Deep State", "CODOH", "Holocaust Denial", "Truth", "Conspiracy", "Awakening",
    "Alternative Media", "UK Column", "Unsafe Space", "VirusTruth", "QAnon",
    "Geo Godley", "Tucker Carlson", "Robert F Kennedy", "E Michael Jones",
    "Stew Peters", "Team Occulted", "Hitler", "Goebbels", "Wehrwolf", "MaeBrussell",
    "Bitchute", "Corona Ausschuss", "Malcolm Roberts", "Carlo Mattogno",
    "DARKHORSEVIDEOS", "Grand-Jury", "Brighteon", "Worldview Weekend", "Peak Prosperity",
    "London Real", "Dr. Reiner Fuellmich", "Harry Vox", "ManFoWars", "Plandemic", "ODD-TV",
    "Anthony C Sutton", "Stephen Mitford Goodson", "Deluminati Entertainment",
    "Charlie Ward", "Beast Philanthropy", "AustraliaOne", "Billy Te Kahika",
    "SKULLCRACK", "JinxMontalgo", "The Truth Above All", "The House of Jacob", "Homeboy's Got Receipts",
    "not my own material!", "Democrats cheated here is the proof", "That Dan Philm Show",
    "Citywide Blackout", "Meet Kevin", "Lineplot Productions", "NullToxicity",

    # Médias / figures très polarisés / controversées
    "Jordan Peterson", "Candace Owens", "David Pakman", "Democracy Now", "OnePath Network",
    "TRT World", "RT ", "Russia Today", "Sputnik", "TASS", "Kremlin",
    "Glenn Beck", "Katherine Austin Fitts", "David Martin",

    # Formats amateurs / auto-promo / farfelus
    "Podcast", "Radio", "Blog", "YouTube", "Soundcloud", "Substack",
    "Studio", "Productions", "Network", "Press", "Publishing", "AuthorScript",
    "puzzle in a thunderstorm", "LLC", "Inc.", "Enterprises", "Consultancy",
    "Agency", "Solutions", "Marketing",
    "360nd Media", "The Enginerdy Show", "Birdtown Audio Poems", "Encryption Europe",
    "Main Devs: Gorser", "Moddroid", "Junkie's Tape Bin", "Polyphony Più", "Studio A.I.S.",

    # Non-pertinent / bruit (artistes, pseudos, divertissement)
    "Cats", "Funny", "Comedy", "Humor", "Satire", "Cartoon", "Meme", "Gaming",
    "Wrestling", "Anime", "Portrait", "Dog", "Books", "Novels", "Art",
    "Creative", "Coffee", "Music", "Butcher", "Netflix", "Clips",
    "Vyzygoth", "Zeph Daniel", "DCP", "mediamode", "monochromec", "Kane Pixels",
    "megatronfucks", "Ghostemane", "Ghostemane And Blackmage", "HeRBaN LyRiX", "AyakoTheClown",
    "BearegardeTheArtist", "u/writerofjots", "u/sofieb30", "u/onlyfuturehuman", "a740g",
    "Pretzelmintz", "Twinkle O.", "VKRISH17", "nipples", "FREMTHEMUKANASATALHARAJAJNABJWALAS"
    ,"Atlantis Software", "SpongeBob & Splaat", "W. Lougee", "Virgin", "UBI Soft", "Renegade", "Image Works", "Loriciel", "Codemasters",
    "Ocean", 'Michael Aldrich', 'Alberta', 'Goldsmith, Olivia', 'Petty, Kate',
       'U.S. Gold', 'First U Think', 'https://berserker.biz/',
       'Dongleware Verlags GmbH', 'Chomsky, Noam', 'Psygnosis',
       'Dr. Doreen Dupont', 'Empire', 'Domark', 'MicroProse', 'Audiogenic',
       'Lankhor', 'Core Design', 'KMO', 'Sloman, John, 1947-',
       'Lanchester, John', 'Roger Lee Bagula', 'Expanding Horizons',
       'Arnold, Roger A', 'Microids',
    #    "['Stanford Classics in Theater', 'Aristophanes']",
       'Backman, Fredrik, 1981- author', 'Banks, Erik', 'Hubbard, R. Glenn',
       'Stiglitz, Joseph E', 'Thalion', 'Interplay', 'Zeppelin', 'Starbyte',
       'Infogrames', 'Krisalis Software', 'Silmarils'
]



pattern = re.compile("|".join(exclude_keywords), flags=re.IGNORECASE)

df_clean = df[~df["creator"].astype(str).str.contains(pattern, na=False)]
counts = df_clean['creator'].value_counts()
df_clean = df_clean[df_clean['creator'].isin(counts[counts > 2].index)]
# pd.DataFrame(df_clean['creator']).to_csv('result2.txt', index= False, header= False, sep = '|')
df_clean


,ticker,title,date,creator,language,collection,subject,rights,description,format,publicdate,link
21,SP500,GeekJuiceMedia's Enter The Dragon ☯ The Final ...,2025-09-28T00:00:00Z,KonCategory3,eng,NaN,NaN,NaN,The Final Game Of Death (2023) Included with ...,NaN,NaN,https://archive.org/details/geek-juice-medias-...
24,SP500,RWDSE - End The Fed,2025-09-28T00:00:00Z,RWDSE,eng,NaN,NaN,NaN,o/,NaN,NaN,https://archive.org/details/rwdse-end-the-fed_...
27,SP500,The Best Of RWDSE,2025-09-28T00:00:00Z,RWDSE,eng,NaN,NaN,NaN,0:00 Hey There Europa 4:28 The Death Squad 6:4...,NaN,NaN,https://archive.org/details/the-best-of-rwdse
35,SP500,Flush Your Meds #flushyourmeds #scottbarry MP4,2025-09-27T00:00:00Z,Scott Barry,eng,NaN,NaN,NaN,Flush Your Meds #flushyourmeds #scottbarry MP4 ;,NaN,NaN,https://archive.org/details/flush_your_meds
36,SP500,Farmers Not The Cause Of Food Price Inflation ...,2025-09-27T00:00:00Z,Redeye Collective,eng,NaN,NaN,NaN,The National Farmers Union says Canadians are ...,NaN,NaN,https://archive.org/details/farmers-not-the-ca...
...,...,...,...,...,...,...,...,...,...,...,...,...
233350,SP500,Department of Labor and Industry and the State...,1990-01-01T00:00:00Z,Montana. Legislature. Office of the Legislativ...,eng,NaN,NaN,NaN,"['""90-5.""', '""March 1991.""']",NaN,NaN,https://archive.org/details/departmentoflabo19...
233363,SP500,Dark Century,1990-01-01T00:00:00Z,Titus,eng,NaN,NaN,NaN,Dark Century (1990)(Titus)[cr Equinox].st,NaN,NaN,https://archive.org/details/Dark_Century_1990_...
233372,SP500,Omnibus Budget Reconciliation Act of 1990 : re...,1990-01-01T00:00:00Z,United States. Congress. House. Committee on t...,eng,NaN,NaN,NaN,['Distributed to some depository libraries in ...,NaN,NaN,https://archive.org/details/omnibusbudgetrec00...
233383,SP500,Economic growth & employment structure : a stu...,1990-01-01T00:00:00Z,"Ghose, Ajit Kumar, 1947-",eng,NaN,NaN,NaN,"['xi, 99 p. ; 24 cm', 'Includes bibliographica...",NaN,NaN,https://archive.org/details/economicgrowthem00...


In [66]:
df_clean = df_clean[df_clean['title'].str.len() > 20]#['creator'].value_counts().index
df_clean[df_clean['title'].str.len() == df_clean['title'].str.len().min()]

,ticker,title,date,creator,language,collection,subject,rights,description,format,publicdate,link
830,SP500,Sick & Swell Part One,2025-08-17T00:00:00Z,Growth-Under-Control,eng,NaN,NaN,NaN,G-U-C's Sick & Swell series part one. Currentl...,NaN,NaN,https://archive.org/details/sick-swell-part-one
1045,SP500,Bankrupt: TGI Fridays,2025-08-01T00:00:00Z,Jake Williams,eng,NaN,NaN,NaN,An episode of Bright Sun Films' Bankrupt serie...,NaN,NaN,https://archive.org/details/bankrupt-tgi-fridays
4192,SP500,R-Cade NES Pack (NEW),2025-01-28T00:00:00Z,Black Panther & TechZombie,eng,NaN,NaN,NaN,TechZombie & Black Panther proudly present a n...,NaN,NaN,https://archive.org/details/nes_20250129
11776,SP500,Decentered or Erased?,2023-09-21T21:50:39Z,The Foothills Deeper Pod,eng,NaN,NaN,NaN,One of the loudest conversations in our cultur...,NaN,NaN,https://archive.org/details/The_Foothills_Deep...
12767,SP500,I Am Your Daily Bread,2023-07-23T00:00:00Z,Dr. Joshua M. Phillips,eng,NaN,NaN,NaN,Exodus 16 KJV 16 And they took their journey ...,NaN,NaN,https://archive.org/details/i-am-your-daily-bread
15692,SP500,R-Cade PC Engine Pack,2023-02-05T00:00:00Z,Black Panther & TechZombie,eng,NaN,NaN,NaN,TechZombie & Black Panther proudly present a c...,NaN,NaN,https://archive.org/details/pcengine_202302
15896,SP500,The Invention of Time,2023-01-25T12:52:36Z,The Foothills Deeper Pod,eng,NaN,NaN,NaN,"Time is in us. We feel time. And yet, what if ...",NaN,NaN,https://archive.org/details/The_Foothills_Deep...
18735,SP500,1 Laamiiɓe 21:1-22:33,2022-08-10T07:00:00Z,Laaawol Dewtere Seniinde,eng,NaN,NaN,NaN,A sermon from Laaawol Dewtere Seniinde.,NaN,NaN,https://archive.org/details/Laaawol_Dewtere_Se...
19258,SP500,FAWP July 9 2022 SHOW,2022-07-09T00:00:00Z,Marilyn Wetston and team,eng,NaN,NaN,NaN,"Marilyn Wetston gives it to you straight ""Fro...",NaN,NaN,https://archive.org/details/fawp-july-9-2022-show
22172,SP500,S&P 500 הגורילות של ה,2022-01-13T03:47:00Z,Laaawol Dewtere Seniinde,eng,NaN,NaN,NaN,"החלק הזה מוקדש ל״גורילות״ של השוק. אפל, אמזון,...",NaN,NaN,https://archive.org/details/Laaawol_Dewtere_Se...


In [70]:
# import de VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # Valence Aware Dictionary and sEntiment Reasoner

# FinBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Charger tokenizer et modèle
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Créer le pipeline de sentiment
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Initialisation de l’analyseur
analyzer = SentimentIntensityAnalyzer()

# FinBert Sentiment Analysis
def finbert_sentiment(text):
    # Handle empty or missing text
    if not isinstance(text, str) or text.strip() == "":
        return 0
    result = finbert(text, truncation = True, max_length = 300)[0]
    label = result["label"].lower()
    confidence = result["score"]

    if label == "positive":
        return confidence
    elif label == "negative":
        return -1 * confidence
    else:  # neutral
        return 0
    


C:\Users\Mchindra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [ ]:
data = df_clean.copy()
data['score'] = data['title'].apply(finbert_sentiment)
data.to_csv("sp500_score.csv", sep = '|', index=False)
data.head(10)

,ticker,title,date,creator,language,collection,subject,rights,description,format,publicdate,link,score
21,SP500,GeekJuiceMedia's Enter The Dragon ☯ The Final ...,2025-09-28T00:00:00Z,KonCategory3,eng,NaN,NaN,NaN,The Final Game Of Death (2023) Included with ...,NaN,NaN,https://archive.org/details/geek-juice-medias-...,0.000000
24,SP500,RWDSE - End The Fed,2025-09-28T00:00:00Z,RWDSE,eng,NaN,NaN,NaN,o/,NaN,NaN,https://archive.org/details/rwdse-end-the-fed_...,0.000000
27,SP500,The Best Of RWDSE,2025-09-28T00:00:00Z,RWDSE,eng,NaN,NaN,NaN,0:00 Hey There Europa 4:28 The Death Squad 6:4...,NaN,NaN,https://archive.org/details/the-best-of-rwdse,0.998333
35,SP500,Flush Your Meds #flushyourmeds #scottbarry MP4,2025-09-27T00:00:00Z,Scott Barry,eng,NaN,NaN,NaN,Flush Your Meds #flushyourmeds #scottbarry MP4 ;,NaN,NaN,https://archive.org/details/flush_your_meds,0.000000
36,SP500,Farmers Not The Cause Of Food Price Inflation ...,2025-09-27T00:00:00Z,Redeye Collective,eng,NaN,NaN,NaN,The National Farmers Union says Canadians are ...,NaN,NaN,https://archive.org/details/farmers-not-the-ca...,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233350,SP500,Department of Labor and Industry and the State...,1990-01-01T00:00:00Z,Montana. Legislature. Office of the Legislativ...,eng,NaN,NaN,NaN,"['""90-5.""', '""March 1991.""']",NaN,NaN,https://archive.org/details/departmentoflabo19...,0.000000
233363,SP500,Dark Century,1990-01-01T00:00:00Z,Titus,eng,NaN,NaN,NaN,Dark Century (1990)(Titus)[cr Equinox].st,NaN,NaN,https://archive.org/details/Dark_Century_1990_...,0.000000
233372,SP500,Omnibus Budget Reconciliation Act of 1990 : re...,1990-01-01T00:00:00Z,United States. Congress. House. Committee on t...,eng,NaN,NaN,NaN,['Distributed to some depository libraries in ...,NaN,NaN,https://archive.org/details/omnibusbudgetrec00...,0.000000
233383,SP500,Economic growth & employment structure : a stu...,1990-01-01T00:00:00Z,"Ghose, Ajit Kumar, 1947-",eng,NaN,NaN,NaN,"['xi, 99 p. ; 24 cm', 'Includes bibliographica...",NaN,NaN,https://archive.org/details/economicgrowthem00...,0.000000


In [77]:
data.head(20)

,ticker,title,date,creator,language,collection,subject,rights,description,format,publicdate,link,score
21,SP500,GeekJuiceMedia's Enter The Dragon ☯ The Final ...,2025-09-28,KonCategory3,eng,NaN,NaN,NaN,The Final Game Of Death (2023) Included with ...,NaN,NaN,https://archive.org/details/geek-juice-medias-...,0.000000
24,SP500,RWDSE - End The Fed,2025-09-28,RWDSE,eng,NaN,NaN,NaN,o/,NaN,NaN,https://archive.org/details/rwdse-end-the-fed_...,0.000000
27,SP500,The Best Of RWDSE,2025-09-28,RWDSE,eng,NaN,NaN,NaN,0:00 Hey There Europa 4:28 The Death Squad 6:4...,NaN,NaN,https://archive.org/details/the-best-of-rwdse,0.998333
35,SP500,Flush Your Meds #flushyourmeds #scottbarry MP4,2025-09-27,Scott Barry,eng,NaN,NaN,NaN,Flush Your Meds #flushyourmeds #scottbarry MP4 ;,NaN,NaN,https://archive.org/details/flush_your_meds,0.000000
36,SP500,Farmers Not The Cause Of Food Price Inflation ...,2025-09-27,Redeye Collective,eng,NaN,NaN,NaN,The National Farmers Union says Canadians are ...,NaN,NaN,https://archive.org/details/farmers-not-the-ca...,0.000000
44,SP500,The Way Of The Dragon 猛龍過江 猛龙过江 (Bruce Lee vs ...,2025-09-26,KonCategory3,eng,NaN,NaN,NaN,GeekJuceMedia's presentation of The Way of the...,NaN,NaN,https://archive.org/details/the-way-of-the-dra...,0.000000
56,SP500,The Man Leading Christian Nationalism’s Rise U...,2025-09-26,The Wall Street Journal,eng,NaN,NaN,NaN,Original Title: The Man Leading Christian Nati...,NaN,NaN,https://archive.org/details/Zlz_0kwno0c,0.000000
57,SP500,The Big Problem With Mamdani’s City-Run Grocer...,2025-09-26,CNBC,eng,NaN,NaN,NaN,Original Title: The Big Problem With Mamdani’s...,NaN,NaN,https://archive.org/details/mRpS3n_Y68k,-0.999996
66,SP500,The Big Boss ( English Export Versions With 2 ...,2025-09-26,KonCategory3,eng,NaN,NaN,NaN,The Big Boss ( Chinese : 唐山大兄 ; originally tit...,NaN,NaN,https://archive.org/details/the-big-boss-engli...,0.000000
80,SP500,The Complex Tech Behind Growing Diamonds in Mi...,2025-09-25,The Wall Street Journal,eng,NaN,NaN,NaN,Original Title: The Complex Tech Behind Growin...,NaN,NaN,https://archive.org/details/SPAMTs6Ug20,0.000000


In [82]:
data['date'] = pd.to_datetime(data['date'],errors = 'coerce').dt.date
score = data.groupby(['date'])['score'].mean().reset_index()
score['score'].value_counts()
# data

score
 0.000000    3972
 0.333333       4
 0.949722       2
-0.333333       2
 0.500000       2
             ... 
-0.989640       1
-0.499916       1
-0.109197       1
-0.999990       1
-0.033619       1
Name: count, Length: 363, dtype: int64

In [86]:
score.to_csv("../data/score.csv", sep = '|', index = False)